In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [22]:
import keras

In [26]:
import building, training, models, callbacks

In [27]:
from importlib import reload

In [78]:
reload(building)
reload(training)
reload(models)
reload(callbacks)

<module 'callbacks' from '/Users/sergmiller/Documents/code/python/alissa/callbacks.py'>

In [29]:
model = models.model1((40, 300))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context1 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context2 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context3 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context4 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
model_3 (M

In [9]:
emb = building.make_emb('datasets/ru.csv')

In [17]:
train = pd.read_csv('datasets/sents/train_sents')
val = pd.read_csv('datasets/sents/val_sents')
train = train.fillna('')
val = val.fillna('')

In [40]:
val.shape

(7876, 8)

In [79]:
def run_model(train, val, emb, epochs=10, batch_size=32, sample_len=8000, load_file=None, save_file=None):
    if load_file is None:
        model = model1((40, 300))
    else:
        model = load_model(model_name)

        model.compile(
                loss=binary_crossentropy,
                optimizer=Adadelta,
        )
        
    
    batch_size=32
    
    filepath = "weights/weights.{epoch:02d}-{val_loss:.3f}.hdf5"
    
    y_train = (train['6'] == 'good').values
    y_val = (val['6'] == 'good').values
    w_train = train['7']

    model.fit_generator(training.flow(train, emb, batch_size, y_train, w_train),
                        steps_per_epoch=train.shape[0] / batch_size, 
                        epochs=epochs, 
                        shuffle=True,
                        callbacks=[callbacks.TelegramCallback(), 
                                   callbacks.RocCallback(train.sample(sample_len, random_state=101), val, emb),
                                  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, 
                                    save_best_only=False, save_weights_only=False, mode='auto', period=1)
                                  ],
                        validation_data=training.flow(val, emb, batch_size, y_train),
                        validation_steps=val.shape[0] / batch_size,
                       )

    if save_file is not None:
        model.save(save_file)

In [81]:
run_model(train[:100], val[:100], emb, sample_len=50)

Epoch 1/10
4/3 [======================================] - 5s 1s/step - loss: 1.1105 - val_loss: 0.7303
Epoch 2/10
4/3 [======================================] - 2s 543ms/step - loss: 0.6642 - val_loss: 0.7019
Epoch 3/10
4/3 [======================================] - 2s 577ms/step - loss: 0.4022 - val_loss: 0.7325
Epoch 4/10
4/3 [======================================] - 2s 549ms/step - loss: 0.2582 - val_loss: 0.8767
Epoch 5/10
4/3 [======================================] - 3s 770ms/step - loss: 0.2510 - val_loss: 1.3797
Epoch 6/10
4/3 [======================================] - 2s 564ms/step - loss: 0.1953 - val_loss: 0.9376
Epoch 7/10
4/3 [======================================] - 2s 533ms/step - loss: 0.0611 - val_loss: 0.9864
Epoch 8/10
4/3 [======================================] - 2s 551ms/step - loss: 0.0384 - val_loss: 1.1165
Epoch 9/10
4/3 [======================================] - 2s 532ms/step - loss: 0.0222 - val_loss: 1.1018
Epoch 10/10
4/3 [================================